# SHOPRun and SHOPCase

1. Explore SHOPRun
2. Create SHOPCase
3. Trigger SHOPRun from SHOPCase
4. Edit Case File
5. Edit Case Data in Python

## Preparation

Instantiating `PowerOpsClient`...

In [1]:
# You can control which setting files are loaded through the environmental variable below.
# In this case, the setting files are located two levels above, in the root of the repository.
import os
from cognite.powerops import PowerOpsClient

os.environ["SETTINGS_FILES"] = "../../settings.toml;../../.secrets.toml"

powerops = PowerOpsClient.from_settings()

## Explore SHOPRun

`SHOPRun` represents a single attempt at running SHOP.

In [2]:
# Retrieve an existing shop run by event external_id.
shop_run_0 = powerops.shop.retrieve("POWEROPS_SHOP_RUN_2023-10-26T09:56:27.964670Z_efc805")
shop_run_0.check_status()

<SHOPRunStatus.SUCCESS: 'success'>

In [3]:
print(shop_run_0)

{
    "external_id": "POWEROPS_SHOP_RUN_2023-10-26T09:56:27.964670Z_efc805",
    "watercourse": "Fornebu",
    "start": "2022-09-20 22:00:00+00:00",
    "end": "2022-10-02 22:00:00+00:00",
    "case_file_external_id": "cog_shop_preprocessor/cogShop/Case_2023-10-26T09:56:27.964670Z_efc805/44f8e8a9-e7ad-4bff-b544-99d8e7637a9c",
    "shop_files_external_ids": [
        {
            "external_id": "SHOP_Fornebu_water_value_cut_file_reservoir_mapping",
            "file_type": "ascii"
        },
        {
            "external_id": "SHOP_Fornebu_water_value_cut_file",
            "file_type": "ascii"
        }
    ],
    "shop_version": "14.4.3.0"
}


In [4]:
list(shop_run_0.get_shop_files())
# also available:
# shop_run_0.get_case_file()
# list(shop_run_0.get_log_files())

['NAMELIST\r\n\r\n# {reservoir name in SHOP} {corresponding module number in ProdRisk}\r\n\r\nDanielsen 201\r\n\r\nLundvann 203\r\n\r\nTangvall 203\r\n\r\nHovden 205\r\n\r\nLivincovs 206\r\n\r\nLensvik 209\r\n\r\nDalbysvatn 210\r\n\r\nRullsvatn 211\r\n\r\nHagen 212\r\n\r\nStrand 213\r\n\r\nStrand_tunnel 213\r\n\r\nRanemsletta_1183 214\r\n\r\nVarhaug 301\r\n\r\nFrosta 302\r\n\r\nNielsen 303\r\n\r\nSirefelt 304\r\n\r\n-1\r\n',
 ' SHOP_WATER_VALUES\r\n         500          15\r\n           1   18418098.7335484\r\n   485.198458311662        464.294844772844        432.239959468076\r\n  0.000000000000000E+000   363.400756356441        363.400756356441\r\n   319.324967312672        259.184558818354        259.184558818354\r\n   128.391646381071        513.466236177169        352.250256249118\r\n   349.946242221269        232.914340353390       0.000000000000000E+000\r\n  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000\r\n   10.5555575216810       0.000000000000000E+000

## Create SHOPCase

`SHOPCase` is an editable representation of a SHOP case. It is used to prepare data for a SHOP run.

We run SHOP by "triggering" `SHOPCase`. This creates a new `SHOPRun` instance.

`SHOPCase` can be created from an existing `SHOPRun` instance (using `shop_run.as_case()`) or by directly instantiating the class. The former is probably useful more often.

In [5]:
# Re-run SHOP using the same data.
case_1 = shop_run_0.to_case()
shop_run_1 = powerops.shop.trigger_case(case_1)
shop_run_1.check_status()

<SHOPRunStatus.IN_PROGRESS: 'in_progress'>

In [6]:
# A small helper function to wait for a shop run to finish.
from time import sleep


def wait(shop_run):
    while shop_run.check_status() == "in_progress":
        sleep(1)
        print(".", end="")
    print("DONE")


wait(shop_run_1)
shop_run_1.check_status()

............................DONE


<SHOPRunStatus.SUCCESS: 'success'>

## Edit Case File

We will save the case YAML file to local filesystem, then make some changes to it and use the changed file to run SHOP again. 

The case file is here edited using Python, but in real use it is expected that the file be edited using external tools.

In [7]:
# Make a new SHOPCase and save the case file to local filesystem.
case = shop_run_1.to_case()
case.save_yaml("case_file.yaml")


# Shop top of the file:
def shop_top_of_file(file_name):
    print(open(file_name).read(200) + "...")


shop_top_of_file("case_file.yaml")

commands:
- set time_delay_unit MINUTE
- set ramping /on
- set stop_cost_from_start_cost /on
- set bypass_loss /on
- set mipgap 0.001000
- set timelimit 600.000
- set reserve_ramping_cost 1
- set fcr_...


In [8]:
# Deliberately break the YAML structure of the file.
# Here we delete first 15 lines of the file for demonstration purposes.
with open("case_file.yaml") as case_file:
    lines = case_file.readlines()
with open("case_file.yaml", "w") as case_file:
    case_file.writelines(lines[15:])
shop_top_of_file("case_file.yaml")

- set code /inc
- start sim 4
connections:
- connection_type: connection_bypass
  from: Hagen
  to: b_Hagen_Strand
- connection_type: connection_bypass
  from: Sirefelt
  from_type: reservoir
  to: b_...


In [9]:
# This is now an invalid YAML file, so we cannot use it with SHOPCase.
try:
    case.load_case_file("case_file.yaml")
except Exception as exc:
    print(repr(exc))

ValueError('Could not parse case data')


In [10]:
# Add "commands:" line to the beginning of the file.
# This will fix the YAML structure, but the case itself will remain broken.
broken_case = open("case_file.yaml").read()
with open("case_file.yaml", "w") as case_file:
    case_file.write("commands:\n")
    case_file.write(broken_case)
shop_top_of_file("case_file.yaml")

commands:
- set code /inc
- start sim 4
connections:
- connection_type: connection_bypass
  from: Hagen
  to: b_Hagen_Strand
- connection_type: connection_bypass
  from: Sirefelt
  from_type: reservoi...


In [11]:
# Now we have a technically valid YAML, we can use it to run SHOP.
case.load_case_file("case_file.yaml")
shop_run_2 = powerops.shop.trigger_case(case)
shop_run_2.check_status()

<SHOPRunStatus.IN_PROGRESS: 'in_progress'>

In [12]:
# We expect SHOP run to run and fail, though.
wait(shop_run_2)
shop_run_2.check_status()

................DONE


<SHOPRunStatus.FAILED: 'failed'>

In [13]:
# More information about the failure can be obtained from the SHOP run object.
shop_run_2.get_failure_info()

{'error': 'Traceback (most recent call last):\n  File "/workspace/cogshop/status_events.py", line 84, in wrapper\n    res = func(client, event_external_id, *args, **kwargs)\n  File "/workspace/cogshop/main.py", line 70, in _main\n    raise CogShopError(cog_shop.solver_status)\ncogshop.exceptions.CogShopError: Integer infeasible\n',
 'failures': None}

## Edit data in Python

Besides editing the case file and using `case.load_case_file()`, we can also interact with case data directly, using `case.data` dict.

We will fix the case data by adding the missing commands back.

In [14]:
# Add missing commmands back, direclty into case data.
case.data["commands"] = [
    "set time_delay_unit MINUTE",
    "set ramping /on",
    "set stop_cost_from_start_cost /on",
    "set bypass_loss /on",
    "set mipgap 0.001000",
    "set timelimit 600.000",
    "set reserve_ramping_cost 1",
    "set fcr_n_equality /on",
    "set reserve_slack_cost 1",
    "set reserve_min_capacity 0.02",
    "set dyn_seg /on",
    "set dyn_juncloss /on",
    "penalty flag /on /plant /schedule",
    "start sim 3",
    "set code /inc",
    "start sim 4",
]

In [15]:
# This time we expect the SHOP run to succeed.
shop_run_3 = powerops.shop.trigger_case(case)
wait(shop_run_3)
shop_run_3.check_status()

.............................DONE


<SHOPRunStatus.SUCCESS: 'success'>

## Cleanup

In [16]:
# Remove the local case file.
import os

os.unlink("case_file.yaml")

End of notebook.